In [1]:
import makeHtml
import requests

class Tistory:
    def __init__(self):
        self.defaultParam = {
            'access_token' : '',
            'output' : 'json'
        }
        from secrets import Secret
        self.secret = Secret()
    def getAuthCode(self):
        # this section will remake at frontend. this is poc code
        # change redirectUrl for realservice url later
        authCodeUrl = "https://www.tistory.com/oauth/authorize"
        params = {
            "client_id": self.secret.tistoryAppId,
            "redirect_uri": "http://ykarma1996.tistory.com/",
            "response_type": "code"
        }
        res = requests.get(authCodeUrl, params=params)
        
        # action this url at browser, and get AccessToken manually just now.
        print (res.url)
        
    def getAccessToken(self, authCode="getFromClient"):
        authCode = input("authCode : ") # delete this later
        
        getTokenUrl = "https://www.tistory.com/oauth/access_token"
        params = {
            "client_id": self.secret.tistoryAppId,
            "client_secret": self.secret.tistorySecretKey,
            "redirect_uri": "http://ykarma1996.tistory.com/",
            "code": authCode,
            "grant_type": "authorization_code"
        }
        res  = requests.get(getTokenUrl, params=params)
        self.defaultParam['access_token'] = res.text.split('=')[1]
    
    def getBlogList(self):
        params = self.defaultParam
        getBlogUrl = "https://www.tistory.com/apis/blog/info"
        res = requests.get(getBlogUrl, params=params)
        
        # Object of key: 'blogName' value: 'blogTitle'
        blogList = {}
        for blog in res.json()['tistory']['item']['blogs']:
            blogList[blog['name']] = blog['title']
        
        # Show to FrontEnd
        print (blogList)
        
    def getcatego(self, blogName):
        params = self.defaultParam
        params['blogName'] = blogName
        getcategoUrl = 'https://www.tistory.com/apis/category/list'
        res = requests.get(getcategoUrl, params=params)
        
        # Object of key : 'catego id' value: 'catego name'
        categoList = {}
        for catego in res.json()['tistory']['item']['categories']:
            categoList[catego['id']] = catego['name']
        
        print (categoList)

    # upload image to imgbb and return image's url
    def uploadImg(self, imgBin):
        imgUpload = "https://api.imgbb.com/1/upload"
        params = {'key': self.imgApiKey}
        imgBase64 = base64.b64encode(imgBin).decode('ascii')
        data = {'image': imgBase64}
        res = requests.post(imgUpload, params=params, data=data).json()
        return res['data']['url']
    
    # replace <img> tag for public viwer
    def rebuildImgStore(self, content):
        for img in content.findAll('img'):
            imgSrc = img.attrs['src']
            res = requests.get(imgSrc, headers=self.headers)
            
            imgUrl = self.uploadImg(res.content)
            # replace <img> tag's src and delete not use attrs
            img.attrs['src'] = imgUrl
            img.attrs['data-image-src'] = ""
            img.attrs['data-base-url'] = ""
        return (str(content))
    
    def postContent(self, blogName, title, visibility, category, tag, acceptComment):
        content = makeHtml.main()
        params = self.defaultParam
        params['blogName'] = blogName
        params['title'] = title
        params['content'] = content
        params['visibility'] = visibility
        params['category'] = category
        params['tag'] = tag
        params['acceptComment'] = acceptComment
        
        postUrl = 'https://www.tistory.com/apis/post/write'
        res = requests.post(postUrl, data=params)
        
        print (res.json()['tistory']['url'])

def main():
    ti = Tistory()
    ti.getAuthCode()
    ti.getAccessToken()
    ti.getBlogList()
    
    #categoName = intput()
    blogName = 'ykarma1996'
    ti.getcatego(blogName)
    
    title = input('게시물 제목 : ')
    visibility = 0
    category = input('카테고리 : ')
    tag = input('테그 : ')
    acceptComment = 1
    
    ti.postContent(blogName, title, visibility, category, tag, acceptComment)
    
if __name__ == "__main__":
    main()

https://www.tistory.com/auth/login/?popup=1&redirectUrl=https%3A%2F%2Fwww.tistory.com%2Foauth%2Fauthorize%3Fclient_id%3D1248d0d1dca9f950e085f8cff4bda0f4%26redirect_uri%3Dhttp%253A%252F%252Fykarma1996.tistory.com%252F%26response_type%3Dcode
authCode : 2bbf71fb49161768abbf67bb490ddd0cd75013b08787543e4f48212345e9480df2b3ad55
{'ykarma1996': '모두의 근삼이', 'moduboan': '모두의 보안'}
{'1107725': 'Marathon_Mesos', '982641': '리눅스', '1011834': '모두의 보안', '1039593': '모두의 보안 1기', '1184515': '개발 이것저것', '1107726': 'Kubernetes', '1039594': '모두의 보안 2기', '1107727': 'Jenkins', '1089407': 'DevOps', '1039595': '모두의 보안 3기', '1107728': 'Docker', '1107730': '미분류', '1107729': 'Ansible', '1011837': 'SYSTEM', '1107731': 'Database', '1011835': 'WEB', '512220': 'IT정보', '826015': '네트워크', '512216': '블로그', '512317': 'Free Chat', '512221': 'Diary'}
게시물 제목 : test123
카테고리 : 1107730
테그 : 123
https://auth.atlassian.com/login?state=g6Fo2SA4VVA0VS0zYW9MWGMtT2VINVo0eHVNT3dINFQ4WnZlSKN0aWTZIGNLZmg4VHlJWkhOd1Y4TkxTZnVpN2FUU0cxUDFfRVl5

https://ykarma1996.tistory.com/121


In [ ]:
ti = Tistory()
ti.getAuthCode()
ti.getAccessToken()
ti.getBlogList()

#categoName = intput()
blogName = 'ykarma1996'
ti.getcatego(blogName)

title = input('게시물 제목 : ')
visibility = 0
category = input('카테고리 : ')
tag = input('테그 : ')
acceptComment = 1

ti.postContent(blogName, title, visibility, category, tag, acceptComment)



https://www.tistory.com/auth/login/?popup=1&redirectUrl=https%3A%2F%2Fwww.tistory.com%2Foauth%2Fauthorize%3Fclient_id%3D1248d0d1dca9f950e085f8cff4bda0f4%26redirect_uri%3Dhttp%253A%252F%252Fykarma1996.tistory.com%252F%26response_type%3Dcode
authCode : 09f22762ece0b65d5bc443bb7ba030855894cedf2d3d4e2dae251dc1781a8f0aea065cd6
{'ykarma1996': '모두의 근삼이', 'moduboan': '모두의 보안'}
{'1107725': 'Marathon_Mesos', '982641': '리눅스', '1011834': '모두의 보안', '1039593': '모두의 보안 1기', '1184515': '개발 이것저것', '1107726': 'Kubernetes', '1039594': '모두의 보안 2기', '1107727': 'Jenkins', '1089407': 'DevOps', '1039595': '모두의 보안 3기', '1107728': 'Docker', '1107730': '미분류', '1107729': 'Ansible', '1011837': 'SYSTEM', '1107731': 'Database', '1011835': 'WEB', '512220': 'IT정보', '826015': '네트워크', '512216': '블로그', '512317': 'Free Chat', '512221': 'Diary'}
게시물 제목 : test1
카테고리 : 1107730
테그 : asd
https://auth.atlassian.com/login?state=g6Fo2SBvWjdGUGVnUEVKS1RtTkliU2JQZGpZSTB0OGtuRGx1QaN0aWTZIGVldlJadU0ydzJpZTd4RG14Rk5zaERIN0NXSXpFaWtEo2

In [5]:
print (ti.defaultParam)

{'access_token': 'invalid_request&error_description', 'output': 'json'}
